In [1]:
import numpy as np
import matplotlib.pyplot as plt

y = np.loadtxt("../data/MADELON/madelon_train.labels", delimiter=',')
x = np.loadtxt("../data/MADELON/madelon_train.data")
#y = np.loadtxt("d:/datasets/uci/madelon/madelon_train.labels", delimiter=',')
#x = np.loadtxt("d:/datasets/uci/madelon/madelon_train.data")
#y = np.loadtxt("d:/datasets/uci/dexter/dexter_train.labels", delimiter=',')
#x = np.loadtxt("d:/datasets/uci/dexter/dexter_train.csv", delimiter=',')
mx=np.mean(x,axis=0)
sx=np.std(x,axis=0)
sx[sx==0]=0.00001
x=(x-mx)/sx
n,d=x.shape
y=y[:, np.newaxis]
X=x.copy()
print(X.shape)

(2000, 500)


In [2]:
yt = np.loadtxt("../data/MADELON/madelon_valid.labels", delimiter=',')
xt = np.loadtxt("../data/MADELON/madelon_valid.data")
#yt = np.loadtxt("d:/datasets/uci/madelon/madelon_valid.labels", delimiter=',')
#xt = np.loadtxt("d:/datasets/uci/madelon/madelon_valid.data")
#yt = np.loadtxt("d:/datasets/uci/dexter/dexter_valid.labels", delimiter=',')
#xt = np.loadtxt("d:/datasets/uci/dexter/dexter_valid.csv", delimiter=',')
xt=(xt-mx)/sx
#yt[yt<0]=0
yt=yt[:, np.newaxis]
yt.shape

(600, 1)

In [3]:
ys=y.copy()
y[y<0]=0
ys=ys[:, np.newaxis]
y.shape

(2000, 1)

In [4]:
ys=ys[:, np.newaxis]
ys.shape

(2000, 1, 1, 1)

In [5]:
def err(x,y,w):
    xw=np.matmul(x,w)
    yxw=y*xw
    return np.mean(yxw<=0)
def loss(x,y,w,s):
    xw=np.matmul(x,w)
    yxw=y*xw
    l=np.log(1+(yxw-1)**2)
    #print(l.shape)
    l[yxw>1]=0
    lo=np.mean(l)
    #print(lo.shape)
    return lo+s*np.sum(w**2)
def gradloss(x,y,w,s):
    xw=np.matmul(x,w)
    yxw=y*xw
    N,d=x.shape
    ly=2*(yxw-1)/(1+(yxw-1)**2)*y
    ydx=np.matmul(ly.T,x)
    g=ydx.T/N+2*s*w
    #print(g.shape)
    return g

In [6]:
x=X.copy()
n,d=x.shape
w=np.zeros((d,1))
s=0.001
eta=0.01
k=10
mu=50
Niter=100
lg=[loss(x,y,w,s)]
idx=np.array(range(0,d))
for i in range(0,Niter):
    g=gradloss(x,y,w,s)
    w=w-eta*g;
    lg.append(loss(x,y,w,s)) 
    Me=int(np.floor(k+(d-k)*max(0.,(Niter-2*i)/(Niter+2.*i*mu))))
    if Me>=x.shape[1]:
        continue

    indc = np.absolute(w)
    sort = np.argsort(indc)
    sort = sort[-Me:]
    w=w[sort]
    x=x.T[sort].T
    xt=xt.T[sort].T
    
    print(k,Me,x.shape[1],w.shape[0])
print(np.sum(np.abs(w)>0))
print("train err=%g, test err=%g\n"%(err(x,ys,w),err(xt,yt,w)))
plt.plot(lg,'k')
plt.savefig('c:/tmp/loss.png', dpi = 300)
plt.show()
plt.close()

10 250 1 250


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 250)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5000, 1, bias=False)

    def forward(self, x):
        x = self.fc1(x)
        return x

net = Net()
print(net.fc1.weight.shape)
net.fc1.weight=nn.Parameter(torch.zeros(1,5000),requires_grad=True)

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
#optimizer = optim.Adam(net.parameters(), lr=0.03)

xx=torch.from_numpy(X.astype(np.float32))
yy=torch.from_numpy(y.astype(np.int))

for epoch in range(30): 
    inputs = xx
    #print(inputs.shape)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    #print(outputs)
    #loss = criterion(outputs, labels)
    u=yy*outputs
    loss = torch.mean(torch.log(1+F.relu(u-1)**2))
    loss.backward()
    optimizer.step()
    #break
    # print statistics
    running_loss = loss.item()
    print('[%d] loss: %.3f' %
             (epoch + 1, running_loss ))

print('Finished Training')

In [ ]:
xw=np.matmul(xt,w)
py=np.sign(xw)
py[py<0]=0
1-np.mean(py==yt)

In [ ]:
py[0:10]

In [ ]:
yt[0:10]